# Cuaderno RETO 1: Exploración de los datos

En este cuaderno se realiza la **exploración inicial del dataset actividad economica** proporcionado
para la práctica de Bases de Datos NoSQL.

El objetivo de este reto es **comprender la estructura, el contenido y la calidad de los datos**
sin realizar todavía tareas de limpieza ni inserción en la base de datos.

Esta exploración permitirá identificar:
- Número de registros y atributos
- Tipos de datos
- Valores nulos
- Posibles claves y relaciones
- Problemas de calidad de los datos


### Contexto del conjunto de datos (Actividad Económica)

El conjunto de datos de **actividad económica** recoge la información relativa a las
actividades económicas desarrolladas en los locales incluidos en el Censo de Locales
y Actividades del Ayuntamiento de Madrid.

Cada registro representa una **actividad asociada a un local**, identificado mediante
el campo `id_local`, lo que implica que un mismo local puede tener asociadas una o
varias actividades económicas.

La actividad se describe mediante una **clasificación jerárquica**, compuesta por
sección, división y epígrafe (`id_seccion`, `id_division`, `id_epigrafe`), junto con
sus correspondientes descripciones textuales, permitiendo identificar el tipo de
actividad económica desarrollada en cada local.

El dataset incluye información territorial y administrativa del local, como distrito,
barrio, dirección y tipo de acceso, así como datos de agrupación cuando el local forma
parte de una agrupación comer


In [ ]:
# Imports necesarios
import pandas as pd 

# Configuración para mostrar todas las filas y columnas sin truncar
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

### Carga del dataset

En primer lugar se carga el fichero CSV.
Este dataset contiene datos relativos a las terrazas de hostelería y restauración 
que constan asociadas a locales del censo de locales y actividades del Ayuntamiento de 
Madrid.

El fichero utiliza el carácter `;` como separador de campos.

In [ ]:
df_actividad_economica = pd.read_csv("../../data/raw/actividadeconomica202312.csv", sep=';')

print(df_actividad_economica.shape)

El resultado anterior indica que en total tenemos 169559 filas y 49 columnas presentes en el dataset.

A continuación vamos a visualizar las primeras filas del dataset.

In [ ]:
# Permite obtener una primera idea del contenido y formato de los datos
df_actividad_economica.head()

A continuación se muestra una muestra aleatoria de registros.
Esto permite observar valores representativos que no necesariamente
aparecen en las primeras filas del fichero.

In [ ]:
df_actividad_economica.sample(5, random_state=42)

### Estructura y tipos de datos

Se analiza la estructura del DataFrame para identificar:
- Nombre de los atributos
- Tipo de dato asignado por pandas
- Número de valores no nulos por columna

In [ ]:
df_actividad_economica.info()

A partir de esta información se puede observar que:
- Existen 2 columnas de tipo object. 
- Existen 4 columnas de tipo float. 
- Existen 19 columnas de tipo int64. 
- Existen 24 columnas de tipo str. 
- Los campos que representan fechas están representados como str.

### Análisis de valores nulos

Se comprueba la existencia de valores nulos en el dataset.
En esta fase **no se eliminan ni corrigen**, únicamente se identifican.

In [ ]:
df_actividad_economica.isnull().sum()[df_actividad_economica.isnull().sum() > 0]

El resultado muestra qué columnas contienen valores nulos y cuántos registros se ven afectados en cada caso. Para este caso podemos ver que hay algunos identificadores (seguro se utilizarán dentro de alguna "relación") y coordenadas sin valores.

### Análisis de columnas de texto

Se identifican las columnas de tipo texto para analizar posibles problemas de calidad, como espacios en blanco innecesarios o formatos inconsistentes.

In [ ]:
text_cols = df_actividad_economica.select_dtypes(include="object").columns
print(text_cols)

A continuación se inspeccionan los valores únicos de una columna de texto representativa para observar posibles inconsistencias.

In [ ]:
df_actividad_economica["desc_barrio_local"].unique()

En los valores mostrados se puede observar la presencia de espacios en blanco
al final de las cadenas de texto, lo que indica que será necesario realizar
un preprocesado mínimo antes de la inserción en la base de datos.

### Identificación de columnas de fecha

Se identifican las columnas que contienen información temporal.
Estas columnas suelen comenzar por el prefijo `fx_`.

In [ ]:
date_cols = [c for c in df_actividad_economica.columns if "fx_" in c.lower()]
print(date_cols)

Las columnas identificadas contienen fechas almacenadas como texto,
por lo que será necesario convertirlas a un formato de fecha adecuado
en fases posteriores para permitir consultas temporales.

### Listado completo de columnas

A continuación, se muestra el listado completo de atributos del dataset para identificar posibles claves y relaciones con otros ficheros.

In [ ]:
# Crea una tabla con dos columnas: "Columna" y "Tipo"
tipos_df = df_actividad_economica.dtypes.reset_index()
tipos_df.columns = ['Columna', 'Tipo de Dato']

tipos_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

### Conclusión del dataset y campos relevantes para el modelo de datos

A partir de la exploración realizada, se identifican aquellos campos que
resultan especialmente relevantes para la construcción del modelo de datos,
ya sea por su papel como identificadores, por permitir relaciones con otros
datasets o por aportar información estructural clave.


In [ ]:
cols_actividad_clave = [
    # Identificación y relación principal
    "id_local",

    # Contexto territorial
    "id_distrito_local",
    "desc_distrito_local",
    "id_barrio_local",
    "desc_barrio_local",
    "id_seccion_censal_local",

    # Clasificación administrativa del local
    "id_tipo_acceso_local",
    "desc_tipo_acceso_local",
    "id_situacion_local",
    "desc_situacion_local",

    # Dirección del edificio
    "id_ndp_edificio",
    "id_vial_edificio",
    "clase_vial_edificio",
    "desc_vial_edificio",
    "num_edificio",
    "cal_edificio",
    "secuencial_local_PC",

    # Dirección de acceso
    "id_ndp_acceso",
    "id_vial_acceso",
    "clase_vial_acceso",
    "desc_vial_acceso",
    "num_acceso",
    "cal_acceso",

    # Agrupaciones
    "id_agrupacion",
    "nombre_agrupacion",
    "id_tipo_agrup",
    "desc_tipo_agrup",
    "id_planta_agrupado",
    "id_local_agrupado",

    # Identificación comercial
    "rotulo",

    # Clasificación de la actividad económica
    "id_seccion",
    "desc_seccion",
    "id_division",
    "desc_division",
    "id_epigrafe",
    "desc_epigrafe",

    # Información temporal
    "fx_datos_ini",
    "fx_datos_fin"
]

df_actividad_economica[cols_actividad_clave].drop_duplicates().sample(10, random_state=42)


El dataset aporta la información fundamental sobre la
naturaleza de la actividad desarrollada en cada local, siendo `id_local` el
campo clave que permite relacionarlo con los datasets de locales, licencias y
terrazas.

Se han seleccionado los campos territoriales y administrativos del local con el
objetivo de mantener la coherencia estructural entre datasets y facilitar la
integración de la información en un modelo de datos común.

La clasificación de la actividad económica se estructura jerárquicamente en
sección, división y epígrafe, lo que permite representar distintos niveles de
detalle de la actividad desarrollada. Esta jerarquía resulta especialmente
relevante para la construcción del modelo de datos, ya que permite analizar
actividades a diferentes niveles de agregación y facilita la definición de
relaciones con otros elementos del sistema.

Los campos de dirección y agrupación se conservan para reflejar la posible
existencia de locales agrupados o con accesos diferenciados, así como para
mantener consistencia con los datasets previamente analizados.

Por último, se incluyen los campos temporales que permiten conocer el periodo de
vigencia de los datos, aspecto relevante para análisis históricos y para la
gestión de la evolución de la actividad económica de los locales.
